In [1]:
import pandas as pd
df = pd.read_csv('Final_Populism.csv')

In [12]:
df

,Summary_Embeddings,Summary,Original_Text,Article_Title,Is_Populist
0,"[-0.029149754, -0.021904264, 0.0021603003, -0....",Donnchadh Ó Laoghaire of Sinn Féin criticized ...,"Sinn Féin spokesperson on Social Protection, D...",No sense in government’s all-in approach to EU...,0
1,"[0.019964693, 0.010486043, 0.027138552, 0.0468...","Donnchadh Ó Laoghaire, Sinn Féin's spokesperso...",Lack of space in our schools must be addressed...,Lack of space in our schools must be addressed...,0
2,"[-0.011785262, -0.005118389, -0.028711798, 0.0...","Donnchadh Ó Laoghaire, a Sinn Féin TD, urged t...",We must safeguard the future of League of Irel...,We must safeguard the future of League of Irel...,0
3,"[-0.046097253, -0.027319714, 0.007871007, -0.0...","Donáth Anna, a Momentum EP-képviselője, az Eur...","Szerző: Momentum Mozgalom | máj 31, 2023 | Mé...",Donáth Anna: Európa a megoldás a magyar oktatá...,0
4,"[-0.055969328, 0.08520521, 0.015029931, 0.0698...",Douglas Hoyos criticizes the Austrian governme...,"Douglas Hoyos: ""Eine Taskforce einzuführen ist...",Douglas Hoyos: Eine Taskforce einzuführen ist ...,0
...,...,...,...,...,...
9892,"[-0.057788845, 0.0046745464, -0.011104433, 0.0...",The text highlights growing protests against E...,Nu de Europese top van regeringsleiders op 14 ...,Veel protestacties rond Euro-Top,1
9893,"[-0.027253903, -0.0062221605, -0.051059105, 0....","The text highlights support for the PVV party,...",PVV +2 zetels!Nederlanders willen hun eigen la...,PVV +2 ZETELS!!,1
9894,"[-0.04918098, 0.00316158, -0.027780691, 0.0190...",The text highlights the Dutch government's mis...,Tienduizenden mensen zijn het slachtoffer gewo...,Aftreden kabinet maakt weg vrij voor een eerli...,1
9895,"[-0.054796062, 0.019674687, -0.033823974, 0.03...",The text highlights the ELAM party's oppositio...,Στις 8 Ιουλίου του 2018 ΔΗΣΥ και ΔΗΚΟ ψήφισαν ...,Άρχισαν οι ηλεκτρονικές εκποιήσεις – Το ΕΛΑΜ σ...,1


In [2]:
df = df[['Summary', 'Original_Text', 'Article_Title','Is_Populist']]

In [ ]:
# pip install -U transformers datasets peft accelerate evaluate rouge-score bitsandbytes

import os, random, numpy as np, pandas as pd, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model
import evaluate

MODEL_NAME = "facebook/bart-base"   
MAX_SRC = 1024
MAX_SUM = 128
DROP_TITLE_PROB = 0.5  
USE_LORA = True


df['Is_Populist'] = df['Is_Populist'].astype(int).clip(0,1)

def make_prompt(title, text, may_drop=False):
    use_title = bool(title) and (not may_drop or random.random() > DROP_TITLE_PROB)
    prefix = (title.strip() + ". ") if use_title else ""
    return (prefix + (text or "")).strip()

def row_to_examples(row, may_drop=False):
    x_sum = "summarize: " + make_prompt(row['Article_Title'], row['Original_Text'], may_drop=may_drop)
    x_cls = "classify_populism: " + make_prompt(row['Article_Title'], row['Original_Text'], may_drop=may_drop)
    return [
        {"task":"summarize", "input": x_sum, "target": row["Summary"]},
        {"task":"classify",  "input": x_cls, "target": str(int(row["Is_Populist"]))}
    ]

from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Is_Populist'])
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['Is_Populist'])

train_examples = [ex for _, r in train_df.iterrows() for ex in row_to_examples(r, may_drop=True)]
val_examples   = [ex for _, r in val_df.iterrows()   for ex in row_to_examples(r, may_drop=False)]
test_examples  = [ex for _, r in test_df.iterrows()  for ex in row_to_examples(r, may_drop=False)]

train_ds = Dataset.from_list(train_examples)
val_ds   = Dataset.from_list(val_examples)
test_ds  = Dataset.from_list(test_examples)

val_sum  = val_ds.filter(lambda ex: ex["task"]=="summarize")
val_cls  = val_ds.filter(lambda ex: ex["task"]=="classify")
test_sum = test_ds.filter(lambda ex: ex["task"]=="summarize")
test_cls = test_ds.filter(lambda ex: ex["task"]=="classify")

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.padding_side = "right"  

def tokenize_fn(batch):
    model_inputs = tok(
        batch["input"], max_length=MAX_SRC, truncation=True, padding="max_length"
    )
    labels = tok(
        text_target=batch["target"], max_length=MAX_SUM, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=train_ds.column_names)
val_sum_tok  = val_sum.map(tokenize_fn, batched=True, remove_columns=val_sum.column_names)
val_cls_tok  = val_cls.map(tokenize_fn, batched=True, remove_columns=val_cls.column_names)
test_sum_tok = test_sum.map(tokenize_fn, batched=True, remove_columns=test_sum.column_names)
test_cls_tok = test_cls.map(tokenize_fn, batched=True, remove_columns=test_cls.column_names)


base = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
    attn_implementation="sdpa"  # switch to "eager" if you see kernel issues
).to("cuda")

if USE_LORA:
    peft_cfg = LoraConfig(task_type="SEQ_2_SEQ_LM", r=16, lora_alpha=32, lora_dropout=0.05)
    model = get_peft_model(base, peft_cfg)
else:
    model = base

collator = DataCollatorForSeq2Seq(tokenizer=tok, model=model)

    
    
rouge = evaluate.load("rouge")
acc   = evaluate.load("accuracy"); prec = evaluate.load("precision")
rec   = evaluate.load("recall");   f1   = evaluate.load("f1")

def _decode(pred_ids, label_ids):
    pred_ids  = np.where(pred_ids  != -100, pred_ids,  tok.pad_token_id)
    label_ids = np.where(label_ids != -100, label_ids, tok.pad_token_id)
    preds  = tok.batch_decode(pred_ids,  skip_special_tokens=True)
    labels = tok.batch_decode(label_ids, skip_special_tokens=True)
    return preds, labels

def summarize_metrics(eval_pred):
    pred_ids, label_ids = eval_pred
    preds, labels = _decode(pred_ids, label_ids)
    r = rouge.compute(predictions=preds, references=labels, use_stemmer=True)
    return {f"rouge_{k}": v for k,v in r.items()}

def classify_metrics(eval_pred):
    pred_ids, label_ids = eval_pred
    preds, labels = _decode(pred_ids, label_ids)
    preds_bin  = [1 if (p.strip() and p.strip()[0]=='1') else 0 for p in preds]
    labels_bin = [1 if (l.strip() and l.strip()[0]=='1') else 0 for l in labels]
    return {
        "accuracy":  acc.compute(predictions=preds_bin, references=labels_bin)["accuracy"],
        "precision": prec.compute(predictions=preds_bin, references=labels_bin)["precision"],
        "recall":    rec.compute(predictions=preds_bin, references=labels_bin)["recall"],
        "f1":        f1.compute(predictions=preds_bin, references=labels_bin)["f1"],
    }

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="bart_multitask_lora",
    learning_rate=2e-4,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,        # << small eval batch
    eval_accumulation_steps=32,          # << stream preds to CPU
    gradient_accumulation_steps=4,
    fp16=torch.cuda.is_available(),
    bf16=False,                          # your T1000 won’t do bf16
    save_strategy="epoch",
    save_safetensors=True,
    save_total_limit=3,
    load_best_model_at_end=False,
    metric_for_best_model="eval_rougeLsum",
    greater_is_better=True,
    logging_steps=50,
    dataloader_pin_memory=True,
    gradient_checkpointing=False,
    optim="adamw_torch_fused",
    report_to="none",
    torch_compile=False,

    # critical for memory-safe eval:
    predict_with_generate=True,
    generation_num_beams=1               # keep it cheap
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_sum_tok,            # summarize-task split
    tokenizer=tok,                       # ok to pass tokenizer here
    data_collator=collator,
    compute_metrics=summarize_metrics
)


try:
    model.print_trainable_parameters()
except Exception:
    pass
assert any(p.requires_grad for p in model.parameters()), "No trainable parameters found."


trainer.train()

from torch.utils.data import DataLoader
dl = DataLoader(train_tok.with_format("torch"), batch_size=2, shuffle=True, collate_fn=collator)
b = next(iter(dl))
b = {k: v.to("cuda") for k,v in b.items()}

print("model on:", next(model.parameters()).device)
model.train()
out = model(**b)
print("one-batch loss:", float(out.loss))  
out.loss.backward()
print("grad present:", any(p.grad is not None for p in model.parameters() if p.requires_grad))


# Summarization (val)
trainer.compute_metrics = summarize_metrics
print("Validation – Summarization:", trainer.evaluate(eval_dataset=val_sum_tok, metric_key_prefix="sum_val"))


trainer.evaluate(eval_dataset=val_cls_tok)

trainer.compute_metrics = summarize_metrics
print("Test – Summarization:", trainer.evaluate(eval_dataset=test_sum_tok, metric_key_prefix="sum_test"))

# Classification (test)
trainer.compute_metrics = classify_metrics
print("Test – Classification:", trainer.evaluate(eval_dataset=test_cls_tok, metric_key_prefix="cls_test"))

@torch.no_grad()
def predict_summary_and_label(text: str):
    device = next(model.parameters()).device
    prompt_sum = "summarize: " + text.strip()
    prompt_cls = "classify_populism: " + text.strip()

    in_sum = tok(prompt_sum, return_tensors="pt", truncation=True, max_length=MAX_SRC).to(device)
    in_cls = tok(prompt_cls, return_tensors="pt", truncation=True, max_length=MAX_SRC).to(device)

    out_sum = model.generate(**in_sum, max_length=MAX_SUM)
    out_cls = model.generate(**in_cls, max_length=8)

    summary = tok.decode(out_sum[0], skip_special_tokens=True).strip()
    label_s = tok.decode(out_cls[0], skip_special_tokens=True).strip()
    is_pop  = 1 if (label_s and label_s[0]=='1') else 0
    return summary, is_pop


CUDA: True, bf16: False, dtype: torch.float16


Filter:   0%|          | 0/1980 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1980 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1980 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1980 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/990 [00:00<?, ? examples/s]

/tmp/ipykernel_1649834/1440693320.py:164: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


trainable params: 884,736 || all params: 140,305,152 || trainable%: 0.6306


/home/tyty/anaconda3/envs/Palantir_Project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


In [ ]:
@torch.no_grad()
def predict_summary_and_label(text: str):
    # User only provides Original_Text
    prompt_sum = "summarize: " + text.strip()
    prompt_cls = "classify_populism: " + text.strip()

    in_sum = tok(prompt_sum, return_tensors="pt", truncation=True, max_length=MAX_SRC).to(model.device)
    in_cls = tok(prompt_cls, return_tensors="pt", truncation=True, max_length=MAX_SRC).to(model.device)

    out_sum = model.generate(**in_sum, max_length=MAX_SUM)
    out_cls = model.generate(**in_cls, max_length=8)

    summary = tok.decode(out_sum[0], skip_special_tokens=True).strip()
    label_s = tok.decode(out_cls[0], skip_special_tokens=True).strip()
    is_pop  = 1 if (label_s and label_s[0]=='1') else 0
    return summary, is_pop
